In [1]:
import pandas as pd
import numpy as np
import os, sys
sys.path.insert(0, os.path.abspath('../..'))
import source.data_handling.data_preparation as dp
import source.trading_strategies.trading_strategy as tsh

import numpy as np
import pandas as pd


In [2]:
pred_df = pd.read_csv("../../Data/ModelData/Classification/pred_df_loop_logit.csv", index_col=0, parse_dates=True)
bt = tsh.backtest_paper_regime_switch(
    pred_df,
    price_col="M1WO_O",
    regime_col="y_pred",
    ts_col="timestamp",
    bear_label=0,
)
summary = tsh.compare_regime_strategies(bt, periods_per_year=252, gamma=5.0, benchmark="BuyHoldEq")
print(summary)

             TotalReturn      CAGR    AnnVol    Sharpe  MaxDrawdown  \
Strategy                                                              
Strategy        0.963304  0.042023  0.092020  0.493502    -0.246743   
BuyHoldEq       4.025733  0.103532  0.148473  0.738206    -0.340332   
BuyHoldRF       0.000000  0.000000  0.000000       NaN     0.000000   
Static50_50     1.346223  0.053413  0.074237  0.738206    -0.183277   

             AnnUtility  Δu vs BuyHoldEq  Δu vs HA  Δu vs 50%  Δu vs 100%  
Strategy                                                                   
Strategy       0.024243        -0.030250       NaN  -0.016781    -0.03025  
BuyHoldEq      0.054493         0.000000       NaN        NaN         NaN  
BuyHoldRF      0.000000        -0.054493       NaN        NaN         NaN  
Static50_50    0.041024        -0.013469       NaN        NaN         NaN  


/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/ipykernel_46959/2869902700.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pred_df = pd.read_csv("../../Data/ModelData/Classification/pred_df_loop_logit.csv", index_col=0, parse_dates=True)


In [8]:
pred_df = pd.read_csv("../../Data/ModelData/Classification/pred_df_loop_rf.csv", index_col=0, parse_dates=True)
bt = tsh.backtest_paper_regime_switch(
    pred_df,
    price_col="M1WO_O",
    regime_col="y_pred",
    ts_col="timestamp",
    bear_label=0,
)
summary = tsh.compare_regime_strategies(bt, periods_per_year=252, gamma=5.0, benchmark="BuyHoldEq")
print(summary)

             TotalReturn      CAGR    AnnVol    Sharpe  MaxDrawdown  \
Strategy                                                              
Strategy        0.799490  0.030587  0.101843  0.346917    -0.374235   
BuyHoldEq       3.223124  0.076673  0.166859  0.526556    -0.578172   
BuyHoldRF       0.000000  0.000000  0.000000       NaN     0.000000   
Static50_50     1.200306  0.041270  0.083429  0.526556    -0.339600   

             AnnUtility  Δu vs BuyHoldEq  
Strategy                                  
Strategy       0.009401        -0.008855  
BuyHoldEq      0.018256         0.000000  
BuyHoldRF      0.000000        -0.018256  
Static50_50    0.026529         0.008273  


/var/folders/q9/d545ygdx5rj6lg_00gpykppw0000gn/T/ipykernel_15387/4063035673.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pred_df = pd.read_csv("../../Data/ModelData/Classification/pred_df_loop_rf.csv", index_col=0, parse_dates=True)


In [5]:
df = pd.read_csv("../../Data/MSCI_World_Data.csv")

In [6]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["year"] = df["timestamp"].dt.year
df.groupby("year")["M1WO"].agg(["count"])

,count
year,
2000,259
2001,261
2002,261
2003,261
2004,262
2005,260
2006,260
2007,261
2008,262
